In [1]:
import tensorflow as tf
import numpy as np

class GeradorRedeNeural:
    def __init__(self, input_shape=(64, 64, 3), min_layers=3, max_layers=10):
        self.input_shape = input_shape
        self.min_layers = min_layers
        self.max_layers = max_layers

    def gerar_modelo(self):
        model = tf.keras.Sequential()
        
        # Adiciona a camada de entrada
        model.add(tf.keras.layers.InputLayer(input_shape=self.input_shape))
        
        # Número aleatório de camadas
        num_layers = np.random.randint(self.min_layers, self.max_layers + 1)
        
        for _ in range(num_layers):
            self._adicionar_camada_aleatoria(model)
        
        # Adiciona uma camada de saída densa com 10 unidades (pode ser ajustado conforme necessário)
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(10, activation='softmax'))
        
        return model

    def _adicionar_camada_aleatoria(self, model):
        # Escolhe aleatoriamente o tipo de camada
        layer_type = np.random.choice(['conv', 'dense', 'maxpool'])
        
        if layer_type == 'conv':
            self._adicionar_camada_conv(model)
        elif layer_type == 'dense':
            self._adicionar_camada_densa(model)
        else:  # maxpool
            self._adicionar_camada_maxpool(model)
        
        # Adiciona Dropout aleatoriamente
        if np.random.rand() > 0.5:
            model.add(tf.keras.layers.Dropout(rate=np.random.uniform(0.1, 0.5)))

    def _adicionar_camada_conv(self, model):
        filters = np.random.choice([32, 64, 128, 256])
        kernel_size = np.random.choice([3, 5])
        model.add(tf.keras.layers.Conv2D(filters, (kernel_size, kernel_size), activation='relu'))

    def _adicionar_camada_densa(self, model):
        units = np.random.choice([64, 128, 256, 512])
        model.add(tf.keras.layers.Dense(units, activation='relu'))

    def _adicionar_camada_maxpool(self, model):
        pool_size = np.random.choice([2, 3])
        model.add(tf.keras.layers.MaxPooling2D((pool_size, pool_size)))

gerador = GeradorRedeNeural()
modelo = gerador.gerar_modelo()
modelo.summary()

2024-09-23 13:42:17.102074: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 13:42:17.116574: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 13:42:17.120657: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-23 13:42:17.131086: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-23 13:42:18.050672: W tensorflow/compiler/tf2

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 60, 60, 64)     │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 56, 56, 32)     │        51,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 28, 28, 512)    │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │     1,003,530 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,076,522 (4.11 MB)

 Trainable params: 1,076,522 (4.11 MB)

 Non-trainable params: 0 (0.00 B)

In [2]:
modelo2 = gerador.gerar_modelo()
modelo2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 64, 64, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4, 4, 64)       │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4, 4, 64)       │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4, 4, 128)      │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 4, 4, 128)      │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │        20,490 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 131,850 (515.04 KB)

 Trainable params: 131,850 (515.04 KB)

 Non-trainable params: 0 (0.00 B)